In [1]:
import db_helpers as dbh
import db_parse as dbp

import json
import pandas as pd
from pprint import pprint
import os

In [2]:
db_name = "dos"
schema = "schema.sql"
revision = "test"
materialization = "ladder"
session = "quic_loss"
data_dir = "/usr/local/dos-mitigation/data"
do_wipe = False # If True, ALL existing data will be overwritten

In [3]:
if do_wipe:
    try:
        dbh.db_drop(db_name)
    except:
        pass
    dbh.db_add(db_name, schema)

# elif db_name not in dbh.db_list():
#     dbh.db_add(db_name, schema)

In [4]:
conn = dbh.Connection("dos")
print("Connected to database: {}".format(conn.db_name()))

Connected to database: dos


In [5]:
revision_row = {
    "hash": "5bcc425177d073587dc0f78bf9acf084676d2061",
    "metal": True,
    "model_file": None,
    "nickname": revision
}
if do_wipe:
    conn.insert_dict_as_row('revisions', revision_row)
revision_id = conn.nickname_id("revisions", revision)

In [6]:
materialization_row = {
    "nickname": materialization,
    "revision": revision_id,
    "inventory_file": None
}
if do_wipe:
    conn.insert_dict_as_row('materializations', materialization_row)
materialization_id = conn.nickname_id("materializations", materialization)

In [7]:
print("Parsing session: {}".format(session))

with open("{}/{}/{}/.parameters.json".format(data_dir, materialization, session)) as f:
    parameters = json.load(f)

session_row = {
    "materialization": materialization_id,
    "parameters": parameters,
    "nickname": session
}
conn.insert_dict_as_row("sessions", session_row)
session_id = dbh.result_as_value(conn.db_query("select id from sessions where nickname='{}'".format(session)))

Parsing session: quic_loss


In [8]:
experiment_list = os.listdir("{}/{}/{}".format(data_dir, materialization, session))
experiment_list = [i for i in experiment_list if not i.startswith('.')]
print(f"Found {len(experiment_list)} experiments")
for experiment in experiment_list:
    if experiment.startswith('.'):
        continue
    dbp.parse_experiment(conn, materialization, session, experiment)
    dbp.analyze_experiment(conn, experiment)

Found 129 experiments
Parsing experiment: 2023_12_14_07_01_25
Parsing experiment: 2023_12_14_01_43_12
Parsing experiment: 2023_12_13_23_43_50
Parsing experiment: 2023_12_14_06_21_38
Parsing experiment: 2023_12_14_05_15_21
Parsing experiment: 2023_12_14_04_48_49
Parsing experiment: 2023_12_14_12_46_15
Parsing experiment: 2023_12_13_15_59_49
Parsing experiment: 2023_12_13_17_19_20
Parsing experiment: 2023_12_13_19_18_36
Parsing experiment: 2023_12_13_13_47_14
Parsing experiment: 2023_12_13_14_00_31
Parsing experiment: 2023_12_14_06_34_55
Parsing experiment: 2023_12_14_13_52_23
Parsing experiment: 2023_12_13_18_52_05
Parsing experiment: 2023_12_14_11_39_52
Parsing experiment: 2023_12_14_03_55_48
Parsing experiment: 2023_12_14_15_11_51
Parsing experiment: 2023_12_14_10_20_17
Parsing experiment: 2023_12_14_14_45_20
Parsing experiment: 2023_12_14_05_55_08
Parsing experiment: 2023_12_13_11_21_18
Parsing experiment: 2023_12_13_20_24_57
Parsing experiment: 2023_12_13_21_57_49
Parsing experiment